# Task A: SVM

## 0. Load the preprocessed data
**We will use the preprocessed dataset after image processing and PCA. It characterizes 2 different classes (tumor or not) based on 200 features.** 

In [1]:
# Import necessary libraries
import pickle
import numpy as np
import pandas as pd

In [2]:
# Load preprocessed data with help of pickle.
with open('DataAfterProcess/images_AfterProcess.pickle', 'rb') as handle:
    X = pickle.load(handle)
    
with open('DataAfterProcess/label_AfterProcess.pickle', 'rb') as handle:
    y = pickle.load(handle)
    
# Check result.
print(X.shape, y.shape) 

(3000, 200) (3000,)


### Train Test Split

**Split data into a training set (90%) and a test set (10%). Note that the test set here comes from the dataset.zip file, not the test.zip file.**

In [3]:
# Import necessary libraries
from sklearn.model_selection import train_test_split

In [4]:
# Split data into a training set and a test set (90% training and 10% testing data).
# Notice that all random state is chosen as fixed in this assignment to ensure reproducibility.
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.1, random_state=3)

# Split training set into a new training set and a validation set (90% training and 10% validation data).
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train,y_train, 
                                                          test_size=0.1, random_state=3)

# Check result.
print(X_train_new.shape, y_train_new.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape) 
print('train set: {} | val set: {} | test set: {}'.format(round(len(y_train_new)/len(X),3), 
                                                          round(len(y_val)/len(X),3),
                                                          round(len(y_test)/len(X),3)))

(2430, 200) (2430,) (270, 200) (270,) (300, 200) (300,)
train set: 0.81 | val set: 0.09 | test set: 0.1


## 1. Train a SVM model
**Firstly, we train a SVM model without hyperparameter tuning in order to compare with model trained from GridsearchCV in next step.**

In [5]:
# Import necessary libraries
from sklearn.svm import SVC

In [6]:
# Call the SVC() model from sklearn and fit the model to the training data.
# Notice that all random state is chosen as fixed in this assignment to ensure reproducibility.
svm = SVC(random_state=0)
svm.fit(X_train_new, y_train_new)

SVC(random_state=0)

### Model validation
**Evaluate the model per validation set by checking confusion matrix, classification report, accuracy score and cohen kappa score.**

In [7]:
# Import necessary libraries
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score

In [8]:
# Get predictions from the model.
y_pred = svm.predict(X_val)

# Print model performance: Confusion matrix, accuracy score, classification report and cohen kappa score.
print('Confusion matrix: \n', confusion_matrix(y_val, y_pred), '\n')
print('Accuracy on validation set: ', accuracy_score(y_val, y_pred), '\n')
print('Classification report: \n', classification_report(y_val, y_pred), '\n') 
print('cohen_kappa score：',cohen_kappa_score(y_val, y_pred))

Confusion matrix: 
 [[ 27  12]
 [  5 226]] 

Accuracy on validation set:  0.937037037037037 

Classification report: 
               precision    recall  f1-score   support

           0       0.84      0.69      0.76        39
           1       0.95      0.98      0.96       231

    accuracy                           0.94       270
   macro avg       0.90      0.84      0.86       270
weighted avg       0.93      0.94      0.93       270
 

cohen_kappa score： 0.7247211227060093


## 2. Hyperparameter tuning: GridsearchCV 
**Let's tune the hyper-parameters of SVM with help of GridsearchCV(). We tune the kernel with different regularizarion parameter C here.**

### Implement GrisearchCV

In [9]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV

In [10]:
# Create a dictionary called param_grid and fill out some parameters for C and kernel.
param_grid = {'C': [0.5,5,50],
              'kernel': ['poly', 'rbf', 'sigmoid']}

In [11]:
# Create a GridSearchCV() object and fit it to the training data.
grid_SVMTaskA = GridSearchCV(SVC(), param_grid=param_grid, refit=True, verbose=2)
grid_SVMTaskA.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END .................................C=0.5, kernel=poly; total time=   0.2s
[CV] END .................................C=0.5, kernel=poly; total time=   0.1s
[CV] END .................................C=0.5, kernel=poly; total time=   0.1s
[CV] END .................................C=0.5, kernel=poly; total time=   0.1s
[CV] END .................................C=0.5, kernel=poly; total time=   0.1s
[CV] END ..................................C=0.5, kernel=rbf; total time=   0.2s
[CV] END ..................................C=0.5, kernel=rbf; total time=   0.2s
[CV] END ..................................C=0.5, kernel=rbf; total time=   0.2s
[CV] END ..................................C=0.5, kernel=rbf; total time=   0.2s
[CV] END ..................................C=0.5, kernel=rbf; total time=   0.2s
[CV] END ..............................C=0.5, kernel=sigmoid; total time=   0.1s
[CV] END ..............................C=0.5, ker

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 5, 50],
                         'kernel': ['poly', 'rbf', 'sigmoid']},
             verbose=2)

### Check best hyperparameters

In [12]:
# Print best hyperparameters and score of grid model
print('Best hyperparameter by GridsearchCV():', grid_SVMTaskA.best_params_, '\n')
print('Best score by using this hyperparameter: %.4f' %(grid_SVMTaskA.best_score_), '\n\n')

# Print mean, std of grid model
means_grid = grid_SVMTaskA.cv_results_['mean_test_score']
stds_grid = grid_SVMTaskA.cv_results_['std_test_score']
params_grid = grid_SVMTaskA.cv_results_['params']
    
for mean, stdev, param in zip(means_grid, stds_grid, params_grid):
    print('For parameter %r: Mean score %.4f (with std %.4f)\n'% (param, mean, stdev))

Best hyperparameter by GridsearchCV(): {'C': 50, 'kernel': 'rbf'} 

Best score by using this hyperparameter: 0.9581 


For parameter {'C': 0.5, 'kernel': 'poly'}: Mean score 0.9007 (with std 0.0083)

For parameter {'C': 0.5, 'kernel': 'rbf'}: Mean score 0.9193 (with std 0.0097)

For parameter {'C': 0.5, 'kernel': 'sigmoid'}: Mean score 0.8415 (with std 0.0030)

For parameter {'C': 5, 'kernel': 'poly'}: Mean score 0.9326 (with std 0.0103)

For parameter {'C': 5, 'kernel': 'rbf'}: Mean score 0.9570 (with std 0.0059)

For parameter {'C': 5, 'kernel': 'sigmoid'}: Mean score 0.8089 (with std 0.0068)

For parameter {'C': 50, 'kernel': 'poly'}: Mean score 0.9407 (with std 0.0103)

For parameter {'C': 50, 'kernel': 'rbf'}: Mean score 0.9581 (with std 0.0042)

For parameter {'C': 50, 'kernel': 'sigmoid'}: Mean score 0.8019 (with std 0.0054)



## 3. Model Evaluation
**Evaluate the model per test set by checking confusion matrix, classification report, accuracy score and cohen kappa score.**

In [13]:
# Get predictions from the grid model.
pred_grid_SVMTaskA = grid_SVMTaskA.predict(X_test)

# Print grid model performance: Confusion matrix, accuracy score and classification report.
print('Confusion matrix: \n', confusion_matrix(y_test, pred_grid_SVMTaskA), '\n')
print('Accuracy on test set: ', accuracy_score(y_test, pred_grid_SVMTaskA), '\n')
print('Classification report: \n', classification_report(y_test, pred_grid_SVMTaskA), '\n')
print('cohen_kappa score：',cohen_kappa_score(y_test, pred_grid_SVMTaskA))

Confusion matrix: 
 [[ 40   4]
 [  2 254]] 

Accuracy on test set:  0.98 

Classification report: 
               precision    recall  f1-score   support

           0       0.95      0.91      0.93        44
           1       0.98      0.99      0.99       256

    accuracy                           0.98       300
   macro avg       0.97      0.95      0.96       300
weighted avg       0.98      0.98      0.98       300
 

cohen_kappa score： 0.9185667752442996


## 3. Save model

In [14]:
# Save trained model with help of pickle.
with open('Model/SVMTaskA.pickle', 'wb') as handle:
    pickle.dump(grid_SVMTaskA, handle)